## Preparing Variants from PharmGKB for variant annotation


#### Import libraries

In [ ]:
import os
import sys
import pandas as pd
import nbconvert

#### Import data

In [ ]:
# Import data
path = "/Users/annamontaner/Documents/BSC3/scratch/cli79/cli79334/projects/other/pharmGKB/01_vcf_generation"
vars = pd.read_csv(os.path.join(path,"input_data", "variant_assembly.csv"), header=0)

In [ ]:
# Check data types
vars.dtypes

In [ ]:
# Check how many rows...
print("Total rows: ", len(vars))
print("Gene missing: ", len(vars[vars["gid"].isna()]))
print("Assembly missing: ", len(vars[vars["assembly"].isna()]))
print("Begin missing: ", len(vars[vars["begin"].isna()]))
print("Ref Allele missing: ", len(vars[vars["ref_allele"].isna()]))
print("Var Allele missing: ", len(vars[vars["var_allele"].isna()]))

#### Clean data

In [ ]:
# Remove rows for which reference or alternative alleles are not available
vars = vars[~vars["ref_allele"].isna()]
vars = vars[~vars["var_allele"].isna()]
print(len(vars))

In [ ]:
# Remove rows with "-"" instead of A/C/G/T
vars = vars[~(vars["ref_allele"]=="-")]
vars = vars[~(vars["var_allele"]=="-")]
print(len(vars))

In [ ]:
print(len(vars[vars["var_hgvs"].isna()]))

In [ ]:
# a) use ref and alt alleles' length 
snvs_tmp = vars[vars['ref_allele'].str.len() == vars['var_allele'].str.len()]

# b) remove variants containing characters other than A/C/G/T
allowed_values = ['A', 'C', 'G', 'T']
snvs = snvs_tmp[snvs_tmp['ref_allele'].str.match(f'^({"|".join(allowed_values)})$') &
               snvs_tmp['var_allele'].str.match(f'^({"|".join(allowed_values)})$')]

"""
Some single-nucleotide changes don't have a proper var_hgvs annotation so
it's better to use the nucleotide length of reference and alt alleles instead.
E.g. 
12441	PA166161042	rs78365220	G6PD	PA28469	GRCh38	154535270.0	154535270.0	0.0	A	NC_000023.11:g.154535270=	C	NaN	[GRCh38]chrX
9934	PA166157504	rs72552784	ABCB1	PA267	GRCh38	87516598.0	87516598.0	0.0	C	NC_000007.14:g.87516598=	A	NC_000006.12:g.160139851_160139853del	[GRCh38]chr7	left_only
"""

In [ ]:
print("Rows vars: ", len(vars))
print("Rows snvs_tmp: ", len(snvs_tmp))
print("Rows snvs: ", len(snvs))

In [ ]:
# Check variant types in snvs dataframe
print('Ref allele types: ',snvs['ref_allele'].unique())
print('Var allele types: ',snvs['var_allele'].unique())

In [ ]:
# Change some column types
snvs['begin'] = snvs['begin'].astype("int")
snvs['chr'] = snvs['chr'].astype("str")
snvs['chr'] = snvs['chr'].str.split("]").str[1].str.replace("chr","")

In [ ]:
# Build final dataframe
df_tmp=snvs.copy()
df_tmp = df_tmp.fillna(".")

In [ ]:
# Build non-existent columns
df_tmp['ID'] = "."
df_tmp['QUAL'] = "."
df_tmp['FILTER'] = "NONE"

# Build info column
info = "PGKBVID="+df_tmp["vid"]+";RSID="+df_tmp['variant']+";GENE="+df_tmp['gene']+";PGKBGID="+df_tmp['gid']+";HGVS="+df_tmp['var_hgvs']
df_tmp['INFO'] = info

In [ ]:
# Rename columns 
df_tmp.rename(columns={"chr":"#CHROM", "ref_allele": "REF", "var_allele": "ALT", "begin": "POS"}, inplace = True)

In [ ]:
# Reorder columns
df = df_tmp[["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]].sort_values(by=['#CHROM','POS'])

In [ ]:
print("Number of variants in unfiltered PharmGKB dataframe: ",len(~vars['begin'].isna()))
print("Unique positions in unfiltered PharmGKB dataframe: ",len(vars['begin'].unique()))
print("Number of variants in filtered PharmGKB dataframe: ",len(~df['POS'].isna()))
print("Unique positions in filtered PharmGKB dataframe: ",len(df['POS'].unique()))

In [ ]:
# Export final .tsv
df.to_csv(os.path.join(path,"output_data","variant_assembly_auto.tsv"), sep="\t", header=True, index=False)

In [ ]:
# Add header and export .csv
output_VCF = os.path.join(path,"output_data","variant_assembly_auto.vcf")
input_TSV = os.path.join(path,"output_data","variant_assembly_auto.tsv")
I = open(input_TSV, "r")
O = open(output_VCF, "w")

O.write("##fileformat=VCFv4.0\n")
O.write("##FILTER=<ID=NONE,Description=\"Not used\">\n")
O.write("##INFO=<ID=PGKBVID,Number=1,Type=String,Description=\"PharmGKB variant id\">\n")
O.write("##INFO=<ID=RSID,Number=1,Type=String,Description=\"dbSnp id\">\n")
O.write("##INFO=<ID=GENE,Number=1,Type=String,Description=\"Gene name\">\n")
O.write("##INFO=<ID=PGKBGID,Number=1,Type=String,Description=\"PharmGKB gene id\">\n")
O.write("##INFO=<ID=HGVS,Number=1,Type=String,Description=\"HGVS id\">\n")
O.write("##FORMAT=<ID=NO,Number=1,Type=String,Description=\"Not used\">\n")

for iLine in I:
	list = iLine.split("\t")
	newLine = '\t'.join(list)
	O.write(newLine)

I.close()
O.close()